In [1]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# import math

# def fetch_div_relative(url):
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
#         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
#         'Accept-Language': 'en-US,en;q=0.5',
#         'Accept-Encoding': 'gzip, deflate',
#         'Connection': 'keep-alive',
#         'Upgrade-Insecure-Requests': '1'
#     }
    
#     try:
#         response = requests.get(url, headers=headers, timeout=2)
#         response.raise_for_status()  # Raises an HTTPError for bad responses
#     except requests.exceptions.Timeout:
#         print(f"Request timed out for URL: {url}")
#         return None
#     except requests.exceptions.RequestException as e:
#         print(f"Request failed: {e}")
#         return None

#     soup = BeautifulSoup(response.text, 'html.parser')
#     div_relative_content = soup.find('div', class_='relative')
#     return div_relative_content

# def extract_tables(div_relative_content):
#     tables = div_relative_content.find_all('table')
#     return tables

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math

def fetch_match_tables(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=1.5)
        response.raise_for_status()  # Raises an HTTPError for bad responses
    except requests.exceptions.Timeout:
        print(f"Request timed out for URL: {url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    # remove the top 2 tables since they are labels
    match_tables = soup.find_all('table')[2:]
    return match_tables

# URL of the webpage to scrape
url = "http://www.tabletennisleague.com/AGTTA/SessionGroupReportArchive/SessionGroupReport2009November17.HTM"
# url = "http://www.tabletennisleague.com/AGTTA/SessionGroupReportArchive/SessionGroupReport2024July23.HTM"
match_tables = fetch_match_tables(url)
# print("Div relative content:", div_relative_content)

if match_tables:
    print(type(match_tables[7]))
    print(match_tables[7])

<class 'bs4.element.Tag'>
<table cols="19" style="margin-right:7%">
<tr style="margin-bottom:-16px">
<td align="left" width="23%">Brent Chen</td>
<td align="right" width="3%">6</td>
<td align="right" width="3%">1</td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%">_</td>
<td align="right" width="3%"><a href="DisplayMatch.ASP?matchid=27249&amp;sessionnumber=9" title="Click to view details of this match">3</a></td>
<td align="right" width="3%"><a href="DisplayMatch.ASP?matchid=27248&amp;sessionnumber=9" title="Click to view details of this match">3</a></td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%">_</td>
<td align="right" width="4%">20</td>
<td align="right" width="4%">0</td>
<td align="right" width="8%">20</td>
<td align="right" width="9%">7

### Main algo

In [10]:
def extract_players_data(players, num_players):
    # Extract data
    players_data = []
    scores_data = []
    ratings_data = []

    for player in players:
        game_details = player.find_all('td')

        players_data.append(game_details[0].get_text(strip=True))
        
        # changes the indices to match the table depending on the number of players in the table
        if (num_players == 3):
            scores_indices = [7, 8, 9]
            ratings_indices = [12, 13, 14]
        elif (num_players == 4):
            scores_indices = [7, 8, 9, 10]
            ratings_indices = [15, 16, 17, 18]
        elif (num_players == 5):
            scores_indices = [6, 7, 8, 9, 10]
            ratings_indices = [14, 15, 16, 17, 18]
        else:
            raise Exception("Number of players in the table is not 3, 4, or 5")
        # Extract how many games the current player won
        scores = [
            0 if game_details[i].get_text(strip=True) == "F" else int(game_details[i].get_text(strip=True)) 
            if game_details[i].get_text(strip=True).lstrip("-").isdigit()
            else math.nan 
            for i in scores_indices
        ]

        # Extract the ratings change of the current player
        ratings = [
            0 if game_details[i].get_text(strip=True) == "F" else int(game_details[i].get_text(strip=True)) 
            if game_details[i].get_text(strip=True).lstrip("-").isdigit() 
            else math.nan 
            for i in ratings_indices
        ]

        scores_data.append(scores)
        ratings_data.append(ratings)
    
    return players_data, scores_data, ratings_data


# URL of the webpage to scrape
url = "http://www.tabletennisleague.com/AGTTA/SessionGroupReportArchive/SessionGroupReport2024May28.HTM"
url = "http://www.tabletennisleague.com/AGTTA/SessionGroupReportArchive/SessionGroupReport2009November17.HTM"
match_tables = fetch_match_tables(url)

session_data = []

for table_num, table in enumerate(match_tables):
    table_num += 1
    players = table.find_all('tr')
    
    players_data, scores_data, ratings_data = extract_players_data(players, len(players))

    for i in range(len(players_data)):
        for j in range(i + 1, len(players_data)):
            if i != j:
                try:
                    session_data.append([players_data[i], players_data[j], scores_data[i][j], scores_data[j][i], abs(ratings_data[i][j]), table_num])
                except IndexError:
                    print("players_data:", players_data)
                    print("scores_data:", scores_data)
                    print("IndexError:", i, j, "table number:", table_num)
                    # print("session_data:", session_data)

session_data

[['N J Ozumba', 'Octavian Prunescu', 3, nan, 25, 1],
 ['N J Ozumba', 'Charles Olayinka', 2, 2, 4, 1],
 ['N J Ozumba', 'Willy Xiao', nan, 1, 2042, 1],
 ['Octavian Prunescu', 'Charles Olayinka', 3, nan, 48, 1],
 ['Octavian Prunescu', 'Willy Xiao', nan, 2, 1975, 1],
 ['Charles Olayinka', 'Willy Xiao', nan, nan, 2086, 1],
 ['Ilya Bushkanets', 'Andrew Dyl', 3, nan, 7, 2],
 ['Ilya Bushkanets', 'Hae Lee', 3, 3, 5, 2],
 ['Ilya Bushkanets', 'Leon Chen', nan, 2, 1896, 2],
 ['Andrew Dyl', 'Hae Lee', 3, nan, 11, 2],
 ['Andrew Dyl', 'Leon Chen', nan, 0, 1881, 2],
 ['Hae Lee', 'Leon Chen', nan, nan, 1893, 2],
 ['Kareem Olugbenga', 'Yoshiyuki Okawa', 3, nan, 10, 3],
 ['Kareem Olugbenga', 'Guodong Zhu', 3, 1, 28, 3],
 ['Kareem Olugbenga', 'Alberto Prieto', nan, 0, 1853, 3],
 ['Yoshiyuki Okawa', 'Guodong Zhu', 3, nan, 5, 3],
 ['Yoshiyuki Okawa', 'Alberto Prieto', nan, 2, 1859, 3],
 ['Guodong Zhu', 'Alberto Prieto', nan, nan, 1812, 3],
 ['James Morris', 'John Karp', 3, nan, 0, 4],
 ['James Morris', 'Bry

## Test Cases

In [ ]:
test_case_3_players_table = """
<table cols="19" style="margin-right:7%">
<tr style="margin-bottom:-16px">
<td align="left" width="23%">Brent Chen</td>
<td align="right" width="3%">6</td>
<td align="right" width="3%">1</td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%">_</td>
<td align="right" width="3%"><a href="DisplayMatch.ASP?matchid=27249&amp;sessionnumber=9" title="Click to view details of this match">3</a></td>
<td align="right" width="3%"><a href="DisplayMatch.ASP?matchid=27248&amp;sessionnumber=9" title="Click to view details of this match">3</a></td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%">_</td>
<td align="right" width="4%">20</td>
<td align="right" width="4%">0</td>
<td align="right" width="8%">20</td>
<td align="right" width="9%">707</td>
<td width="1%"></td>
</tr>
<tr style="margin-bottom:-16px">
<td align="left" width="23%">Mohamed Ghallab</td>
<td align="right" width="3%">4</td>
<td align="right" width="3%">3</td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%"><a href="DisplayMatch.ASP?matchid=27249&amp;sessionnumber=9" title="Click to view details of this match">1</a></td>
<td align="right" width="3%">_</td>
<td align="right" width="3%"><a href="DisplayMatch.ASP?matchid=27247&amp;sessionnumber=9" title="Click to view details of this match">3</a></td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%">-20</td>
<td align="right" width="4%">_</td>
<td align="right" width="4%">0</td>
<td align="right" width="8%">-20</td>
<td align="right" width="9%">774</td>
<td width="1%"></td>
</tr>
<tr style="margin-bottom:-16px">
<td align="left" width="23%">David Wang</td>
<td align="right" width="3%">0</td>
<td align="right" width="3%">6</td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%"></td>
<td align="right" width="3%"><a href="DisplayMatch.ASP?matchid=27248&amp;sessionnumber=9" title="Click to view details of this match">0</a></td>
<td align="right" width="3%"><a href="DisplayMatch.ASP?matchid=27247&amp;sessionnumber=9" title="Click to view details of this match">0</a></td>
<td align="right" width="3%">_</td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%"></td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">_</td>
<td align="right" width="8%">0</td>
<td align="right" width="9%">422</td>
<td width="1%"></td>
</tr>
</table>
"""

# Parse the HTML content
soup = BeautifulSoup(test_case_3_players_table, 'html.parser')
inner_tables = soup.find_all('tr')

# Unit test
players_data, scores_data, ratings_data = extract_players_data(inner_tables)

print("Players Data:", players_data)
print("Scores Data:", scores_data)
print("Ratings Data:", ratings_data)

Exception: Number of players in the table is not 3, 4, or 5

In [ ]:
test_case_4_players_table = """
<table cols="21" style="margin-right:7%">
<tr style="margin-bottom:-16px">
<td class="datacolumn1" style="color:red"><b>Sunil Bandapally</b></td>
<td class="gameswlcolumn">9</td>
<td class="gameswlcolumn">2</td>
<td class="beforegw"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn">_</td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59199&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59197&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59195&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="aftergw"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td align="right" width="4%">_</td>
<td align="right" width="4%">8</td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">2</td>
<td class="nracolumn">10</td>
<td class="frcolumn">2088</td>
</tr>
<tr style="margin-bottom:-16px">
<td class="datacolumn1">Geetha Krishna</td>
<td class="gameswlcolumn">8</td>
<td class="gameswlcolumn">4</td>
<td class="beforegw"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59199&amp;sessionnumber=7" title="Click to view details of this match">2</a></td>
<td class="gamesemcolumn">_</td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59196&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59198&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="aftergw"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td align="right" width="4%">-8</td>
<td align="right" width="4%">_</td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">2</td>
<td class="nracolumn">-6</td>
<td class="frcolumn">2063</td>
</tr>
<tr style="margin-bottom:-16px">
<td class="datacolumn1" style="color:red"><b><a href="http://www.tabletennisleague.com/agtta/playerbio/PlayerBio.ASPx?playerID=1483" title="Click to view bio page for this player">Andrew Dyl</a></b></td>
<td class="gameswlcolumn">4</td>
<td class="gameswlcolumn">8</td>
<td class="beforegw"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59197&amp;sessionnumber=7" title="Click to view details of this match">0</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59196&amp;sessionnumber=7" title="Click to view details of this match">1</a></td>
<td class="gamesemcolumn">_</td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59200&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="aftergw"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">_</td>
<td align="right" width="4%">30</td>
<td class="nracolumn">30</td>
<td class="frcolumn">1788</td>
</tr>
<tr style="margin-bottom:-16px">
<td class="datacolumn1" style="color:red"><b><a href="http://www.tabletennisleague.com/agtta/playerbio/PlayerBio.ASPx?playerID=2649" title="Click to view bio page for this player">Aneece Khalek</a></b></td>
<td class="gameswlcolumn">2</td>
<td class="gameswlcolumn">9</td>
<td class="beforegw"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59195&amp;sessionnumber=7" title="Click to view details of this match">0</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59198&amp;sessionnumber=7" title="Click to view details of this match">0</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59200&amp;sessionnumber=7" title="Click to view details of this match">2</a></td>
<td class="gamesemcolumn">_</td>
<td class="aftergw"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td align="right" width="4%">-2</td>
<td align="right" width="4%">-2</td>
<td align="right" width="4%">-30</td>
<td align="right" width="4%">_</td>
<td class="nracolumn">-34</td>
<td class="frcolumn">1875</td>
</tr>
</table>
"""

# Parse the HTML content
soup = BeautifulSoup(test_case_4_players_table, 'html.parser')
inner_tables = soup.find_all('tr')

# Unit test
players_data, scores_data, ratings_data = extract_players_data(inner_tables)

print("Players Data:", players_data)
print("Scores Data:", scores_data)
print("Ratings Data:", ratings_data)

Players Data: ['Sunil Bandapally', 'Geetha Krishna', 'Andrew Dyl', 'Aneece Khalek']
Scores Data: [[nan, 3, 3, 3], [2, nan, 3, 3], [0, 1, nan, 3], [0, 0, 2, nan]]
Ratings Data: [[nan, 8, 0, 2], [-8, nan, 0, 2], [0, 0, nan, 30], [-2, -2, -30, nan]]


In [ ]:
test_case_5_players_table = """
<table cols="21" style="margin-right:7%">
<tr style="margin-bottom:-16px">
<td class="datacolumn1">Daniel Phillips</td>
<td class="gameswlcolumn">12</td>
<td class="gameswlcolumn">1</td>
<td class="beforegw"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn">_</td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59251&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59249&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59247&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59245&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="aftergw"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td align="right" width="4%">_</td>
<td align="right" width="4%">2</td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">0</td>
<td class="nracolumn">2</td>
<td class="frcolumn">1607</td>
</tr>
<tr style="margin-bottom:-16px">
<td class="datacolumn1">Tresor Avognon</td>
<td class="gameswlcolumn">10</td>
<td class="gameswlcolumn">3</td>
<td class="beforegw"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59251&amp;sessionnumber=7" title="Click to view details of this match">1</a></td>
<td class="gamesemcolumn">_</td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59246&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59250&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59243&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="aftergw"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td align="right" width="4%">-2</td>
<td align="right" width="4%">_</td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">0</td>
<td class="nracolumn">-2</td>
<td class="frcolumn">1435</td>
</tr>
<tr style="margin-bottom:-16px">
<td class="datacolumn1">Pat Tran</td>
<td class="gameswlcolumn">6</td>
<td class="gameswlcolumn">7</td>
<td class="beforegw"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59249&amp;sessionnumber=7" title="Click to view details of this match">0</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59246&amp;sessionnumber=7" title="Click to view details of this match">0</a></td>
<td class="gamesemcolumn">_</td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59244&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59248&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="aftergw"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">_</td>
<td align="right" width="4%">7</td>
<td align="right" width="4%">5</td>
<td class="nracolumn">12</td>
<td class="frcolumn">652</td>
</tr>
<tr style="margin-bottom:-16px">
<td class="datacolumn1">Lukasz Trzuskawski</td>
<td class="gameswlcolumn">3</td>
<td class="gameswlcolumn">10</td>
<td class="beforegw"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59247&amp;sessionnumber=7" title="Click to view details of this match">0</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59250&amp;sessionnumber=7" title="Click to view details of this match">0</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59244&amp;sessionnumber=7" title="Click to view details of this match">0</a></td>
<td class="gamesemcolumn">_</td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59252&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="aftergw"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">-7</td>
<td align="right" width="4%">_</td>
<td align="right" width="4%">7</td>
<td class="nracolumn">0</td>
<td class="frcolumn">604</td>
</tr>
<tr style="margin-bottom:-16px">
<td class="datacolumn1">Leon Fu</td>
<td class="gameswlcolumn">2</td>
<td class="gameswlcolumn">12</td>
<td class="beforegw"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59245&amp;sessionnumber=7" title="Click to view details of this match">0</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59243&amp;sessionnumber=7" title="Click to view details of this match">0</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59248&amp;sessionnumber=7" title="Click to view details of this match">1</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59252&amp;sessionnumber=7" title="Click to view details of this match">1</a></td>
<td class="gamesemcolumn">_</td>
<td class="aftergw"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">0</td>
<td align="right" width="4%">-5</td>
<td align="right" width="4%">-7</td>
<td align="right" width="4%">_</td>
<td class="nracolumn">-12</td>
<td class="frcolumn">559</td>
</tr>
</table>
"""

# Parse the HTML content
soup = BeautifulSoup(test_case_5_players_table, 'html.parser')
inner_tables = soup.find_all('tr')

# Unit test
players_data, scores_data, ratings_data = extract_players_data(inner_tables)

print("Players Data:", players_data)
print("Scores Data:", scores_data)
print("Ratings Data:", ratings_data)

Players Data: ['Daniel Phillips', 'Tresor Avognon', 'Pat Tran', 'Lukasz Trzuskawski', 'Leon Fu']
Scores Data: [[nan, 3, 3, 3, 3], [1, nan, 3, 3, 3], [0, 0, nan, 3, 3], [0, 0, 0, nan, 3], [0, 0, 1, 1, nan]]
Ratings Data: [[nan, 2, 0, 0, 0], [-2, nan, 0, 0, 0], [0, 0, nan, 7, 5], [0, 0, -7, nan, 7], [0, 0, -5, -7, nan]]
